In [28]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import requests

In [29]:
#path = "C:\Program Files (x86)\chromedriver.exe"
path2 = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(path2)
driver.get("https://us.trend-calendar.com/trend/2020-01-01.html")

In [30]:
counter = 365
trend = {}
l = []
while counter != 0:
    counter -= 1
    header = driver.find_elements_by_class_name("entry-title")
    for content in header:
        res = []
        text = content.text
    date = text.split("on ",1)[1]
    contents = driver.find_elements_by_class_name("ranking")
    for content in contents:
        text = content.text
        if text != '' :
            res.append(text)     
    trends = res[0]    
    trend[date] = trends.split("\n")[:5]
    l.append(trends.split("\n")[:5])
    trends = []
    # click submit button
    submit_button = driver.find_elements_by_class_name("yesterdaytomorrow")[1]
    submit_button.click()
    time.sleep(1)

IndexError: list index out of range

In [26]:
#trend

In [21]:
trend['1st January, 2020']

['Winter Classic 2020',
 'Post Malone',
 'Paula Abdul',
 'Premier League',
 'Lucy Hale']

In [32]:
df = pd.DataFrame(l, columns = ['1', '2','3', '4', '5'])
df

,1,2,3,4,5
0,#NewYearsDay,Welcome to 2020,Pope,#RoseBowl,#CitrusBowl
1,New Guy,#ThursdayThoughts,#worldintrovertday,Opie,Anna Mae
2,#wwlll,#TrumpsWar,#WWIII,Rose McGowan,Soleimani
3,Meat Loaf,#ww3memes,#njwk14,Tucker Carlson,#SaturdayThoughts
4,#IfTheWorldHadNoMen,#BTSxGDA,Happy Founders,#PowerTV,#GoldenGlobes
...,...,...,...,...,...
343,Quebec,Lincoln Project,Mark Meadows,#BidenHarris2020,hobi
344,Four Seasons Total Landscaping,Notre Dame,Dave Chappelle,Alex Trebek,Jim Carrey
345,Pfizer,Eva Longoria,Toph,taemin,Brady
346,Xbox,Major League,Pompeo,Veterans Day,#BidenWonDay


In [33]:
#l

In [34]:
df.to_csv('file_name.csv')


"1,2,3,4,5\n#NewYearsDay,Welcome to 2020,Pope,#RoseBowl,#CitrusBowl\nNew Guy,#ThursdayThoughts,#worldintrovertday,Opie,Anna Mae\n#wwlll,#TrumpsWar,#WWIII,Rose McGowan,Soleimani\nMeat Loaf,#ww3memes,#njwk14,Tucker Carlson,#SaturdayThoughts\n#IfTheWorldHadNoMen,#BTSxGDA,Happy Founders,#PowerTV,#GoldenGlobes\nGeorge Lopez,#ARMYSelcaDay,#RuinAMovieWithOneWord,Bolton,#TheBachelor\n#TuesdayThoughts,Nikki Haley,#7isComing,#WithMyNextPaycheckIWill,Meghan McCain\n#WednesdayWisdom,#IvotedforHillaryClinton,#iHeartAwards,#IranVSAmerica,Jillian Michaels\nBoosie,#SniffyMcAdderall,#LawEnforcementAppreciationDay,#ThursdayThoughts,Happy Founders\n#IAmNonbinary,Life is Good,#FridayFeeling,Sum 2 Prove,#RareOutNow\n#Deku,#WorstThingToYellOutBeforeSex,#justinbieberisoverparty,#WitnessesAndDocuments,#SaturdayMorning\n#Deku,#WorstThingToYellOutBeforeSex,#justinbieberisoverparty,#WitnessesAndDocuments,#SaturdayMorning\n#Deku,#WorstThingToYellOutBeforeSex,#justinbieberisoverparty,#WitnessesAndDocuments,#Saturd